Instalando Pacote

In [1]:
!git clone https://github.com/huggingface/transformers;
!cd transformers; pip3 install .

Cloning into 'transformers'...
remote: Enumerating objects: 167258, done.
remote: Counting objects: 100% (74860/74860), done.
remote: Compressing objects: 100% (4320/4320), done.
remote: Total 167258 (delta 72287), reused 70655 (delta 70512), pack-reused 92398
Receiving objects: 100% (167258/167258), 163.81 MiB | 15.37 MiB/s, done.
Resolving deltas: 100% (127920/127920), done.
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.8 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=7923012 sha256=f0236be4915dfc5ab93036cd

In [2]:
! pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.9/243.9 kB 28.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [3]:
! pip install datasets

Puxando Dataset de Discurso de Ódio

In [4]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [5]:
import pandas as pd
import transformers
from transformers import AutoModel, BertTokenizerFast
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer



# link: https://huggingface.co/datasets/ag_news

In [6]:
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/t5_key_text/original/base_offensive_test.csv', sep = ';', decimal = ',')
test.head()

,text,label
0,#ibelieveblaseyford is liar she is fat ugly li...,1
1,@user @user @user I got in a pretty deep debat...,0
2,"...if you want more shootings and more death, ...",0
3,Angels now have 6 runs. Five of them have come...,0
4,#Travel #Movies and Unix #Fortune combined Vi...,0


In [7]:
# X_train = dataset['train']['text']
X_test = test['text']
# y_train = dataset['train']['label']
y_test = test['label']

In [8]:
nclasses = len(np.unique(y_test))

In [9]:
nclasses

2

In [10]:
def puxa_dados_normal(caminho):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')
  original = csv[['texto', 'label']].copy()
  original = original.drop_duplicates()

  # csv = csv[csv['similiaridade_parafrase'] >= 0.95].copy()


  parafrase = csv[['generated', 'label']].copy()
  parafrase = parafrase.drop_duplicates()
  parafrase = parafrase[~(parafrase['generated'].isnull())].copy()
  parafrase.rename(columns={"generated": "texto"}, inplace = True)

  fim = original.append(parafrase)

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values


  return X, y


In [11]:
def puxa_original(caminho):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')

  X = csv['texto'].values
  y = csv['label'].values


  return X, y

In [12]:
def puxa_dados_normal_pkl(caminho):
  csv = pd.read_pickle(caminho)
  original = csv[['texto', 'label']].copy()
  original = original.drop_duplicates()

  # csv = csv[csv['similiaridade_parafrase'] >= 0.95].copy()


  parafrase = csv[['generated', 'label']].copy()
  # parafrase = parafrase.drop_duplicates()
  parafrase = parafrase[~(parafrase['generated'].isnull())].copy()
  parafrase.rename(columns={"generated": "texto"}, inplace = True)

  fim = original.append(parafrase)

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values


  return X, y


In [33]:
def puxa_dados_lambada(caminho, original):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')
  original = pd.read_csv(original, sep = ';', decimal = ',')

  # csv = csv[csv['similiaridade_parafrase'] >= 0.95].copy()


  parafrase = csv[['generated', 'new_label']].copy()
  # parafrase = parafrase.drop_duplicates()
  parafrase = parafrase[~(parafrase['generated'].isnull())].copy()
  parafrase['generated'] = parafrase['generated'].astype('str')
  parafrase.rename(columns={"generated": "texto", "new_label": "label"}, inplace = True)

  fim = original.append(parafrase)
  fim = fim[~(fim['label'].isnull())].copy()

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values


  return X, y

In [14]:
def puxa_dados_lambada_filtro(caminho, original, limiar):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')
  original = pd.read_csv(original, sep = ';', decimal = ',')

  csv = csv[csv['prob_text'] >= limiar].copy()


  parafrase = csv[['generated', 'new_label']].copy()
  parafrase = parafrase.drop_duplicates()
  parafrase = parafrase[~(parafrase['generated'].isnull())].copy()
  parafrase.rename(columns={"generated": "texto", "new_label": "label"}, inplace = True)

  fim = original.append(parafrase)

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values


  return X, y

In [15]:
def puxa_dados_lambada_pkl_filtro(caminho, original, limiar):
  csv = pd.read_pickle(caminho)
  original = pd.read_csv(original, sep = ';', decimal = ',')

  csv = csv[csv['prob_text'] >= limiar].copy()


  parafrase = csv[['generated', 'new_label']].copy()
  parafrase = parafrase.drop_duplicates()
  parafrase = parafrase[~(parafrase['generated'].isnull())].copy()
  parafrase.rename(columns={"generated": "texto", "new_label": "label"}, inplace = True)

  fim = original.append(parafrase)

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values


  return X, y

In [16]:
def desempenho_LSTM(X_train, X_test, y_train, y_test, classes, num_epochs ):


  # vocab_size = 10000
  embedding_dim = 300
  max_length = 128
  trunc_type = 'post'
  padding_type = 'post'
  oov_tok = '<OOV>'

  tokenizer = Tokenizer( oov_token=oov_tok)
  tokenizer.fit_on_texts(X_train)
  word_index = tokenizer.word_index
  vocab_size = len(word_index)+1

  train_sequences = tokenizer.texts_to_sequences(X_train)
  # val_sequences = tokenizer.texts_to_sequences(X_val)

  train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  # val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  # model = tf.keras.Sequential([
  #   # Camada Embedding (reduzir dimensionalidade)
  #   tf.keras.layers.Embedding(vocab_size, embedding_dim),
  #   # Camada de LSTM bidirecional
  #   tf.keras.layers.LSTM(embedding_dim),
  #   # Camada densa com Relu
  #   tf.keras.layers.Dense(embedding_dim, activation='relu'),
  #   #Camada de saída para as 4 classes
  #   tf.keras.layers.Dense(classes, activation='softmax')
  #   ])
  model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size,
                                  mask_zero= True,
                                  output_dim=embedding_dim,
                                  input_length=max_length,
                                  input_shape=(max_length, )),

        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=False)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=classes, activation='softmax')
    ])
  # model = tf.keras.Sequential()
  # model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim))
  # model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  # model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
  # model.add(tf.keras.layers.LSTM(100))
  # model.add(tf.keras.layers.Dense(classes, activation='softmax'))

  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


  history = model.fit(train_padded, y_train, epochs=num_epochs, verbose=2)

  test_sequences = tokenizer.texts_to_sequences(X_test)
  test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


  preds_log = model.predict(test_padded)
  preds = tf.argmax(preds_log, 1)
  acc = accuracy_score(y_test, preds)
  f1 = f1_score(y_test, preds, average='macro')

  return acc, f1







In [17]:
def desempenho_CNN(X_train, X_test, y_train, y_test, classes, num_epochs ):


  # vocab_size = 10000
  embedding_dim = 300
  max_length = 128
  trunc_type = 'post'
  padding_type = 'post'
  oov_tok = '<OOV>'

  tokenizer = Tokenizer( oov_token=oov_tok)
  tokenizer.fit_on_texts(X_train)
  word_index = tokenizer.word_index
  vocab_size = len(word_index)+1

  train_sequences = tokenizer.texts_to_sequences(X_train)
  # val_sequences = tokenizer.texts_to_sequences(X_val)

  train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  # val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  # model = tf.keras.Sequential([
  #   # Camada Embedding (reduzir dimensionalidade)
  #   tf.keras.layers.Embedding(vocab_size, embedding_dim),
  #   # Camada de LSTM bidirecional
  #   tf.keras.layers.LSTM(embedding_dim),
  #   # Camada densa com Relu
  #   tf.keras.layers.Dense(embedding_dim, activation='relu'),
  #   #Camada de saída para as 4 classes
  #   tf.keras.layers.Dense(classes, activation='softmax')
  #   ])
  model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size,
                                  mask_zero= True,
                                  output_dim=embedding_dim,
                                  input_length=max_length,
                                  input_shape=(max_length, )),

        tf.keras.layers.Conv1D(filters=100, kernel_size = 3, activation = 'relu',
                               # set 'axis' value to the first and second axis of conv1D weights (rows, cols)
                               kernel_constraint= MaxNorm( max_value=3, axis=[0,1])),

        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='relu',
                              # set axis to 0 to constrain each weight vector of length (input_dim,) in dense layer
                              kernel_constraint = MaxNorm( max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=classes, activation='softmax')
    ])
  # model = tf.keras.Sequential()
  # model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim))
  # model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  # model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
  # model.add(tf.keras.layers.LSTM(100))
  # model.add(tf.keras.layers.Dense(classes, activation='softmax'))

  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


  history = model.fit(train_padded, y_train, epochs=num_epochs, verbose=2)

  test_sequences = tokenizer.texts_to_sequences(X_test)
  test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


  preds_log = model.predict(test_padded)
  preds = tf.argmax(preds_log, 1)
  acc = accuracy_score(y_test, preds)
  f1 = f1_score(y_test, preds, average='macro')

  return acc, f1







In [18]:
# Evaluate the model
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
      return f1_score(labels, preds, average='macro')


In [19]:
def desempenho_transformers(X_train, X_test, y_train, y_test,classes, model,type_model, epochs, batch, max_len):
  from simpletransformers.classification import ClassificationModel
  import pandas as pd
  import logging
  import sklearn

  train_df = pd.DataFrame(X_train)
  train_df['target'] = y_train

  eval_df = pd.DataFrame(X_test)
  eval_df['target'] = y_test



  # Create a ClassificationModel
  model = ClassificationModel(type_model, model, num_labels=classes, args={'num_train_epochs':epochs,
                                                                           'train_batch_size':batch,
                                                                           'max_seq_length':max_len,
                                                                          #  "fp16":False,
                                                                           'overwrite_output_dir': True})

  # Train the model
  model.train_model(train_df)
  result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score,f1=f1_multiclass)

  return result['acc'], result['f1']





In [20]:

y_test = np.array(y_test)


# Modelos

In [35]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
llms = [
       'gpt_neo_125m',
       'gpt_neo_13b',
       'gpt_neo_27b',
       'gpt_j_6b',
       'bloom_560m',
       'bloom_1b',
       'bloom_3b',
       'bloom_7b',
       'bloomz_560m',
       'bloomz_1b',
       'bloomz_3b',
       'bloomz_7b',
       'llama2_7b',
       'llama2_13b',
       'mistral_7b',
       'mistral_instruct_7b',
       'falcon_7b',
       'falcon_instruct_7b',
       ]

rota = '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/modelo/base_offensive_modelo_'

borig = [
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
        ]


roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(llms)):
  print(j)
  arq = rota+llms[j]+'.csv'
  X_train,  y_train = puxa_dados_lambada(arq, borig[0])


  roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  print('Roberta')
  print('ACC:' + str(roberta[0]))
  print('F1:' + str(roberta[1]))


  roberta_acc.append(roberta[0])
  roberta_f1.append(roberta[1])

0


<ipython-input-33-48d1c59c153e>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1020 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/32 [00:00<?, ?it/s]

...

Running Epoch 3 of 4:   0%|          | 0/32 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

Roberta
ACC:0.7395348837209302
F1:0.6982815329002187


In [36]:
data = {'llms': llms, # 'eda1', 'eda2', 'eda3', 'eda4','eda5'],
        'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,}



base_resultados = pd.DataFrame.from_dict(data)

In [37]:
base_resultados

,llms,ROBERTA_ACC,ROBERTA_F1
0,gpt_neo_125m,0.675581,0.657998
1,gpt_neo_13b,0.733721,0.707161
2,gpt_neo_27b,0.815116,0.756846
3,gpt_j_6b,0.525581,0.523841
4,bloom_560m,0.551163,0.548169
5,bloom_1b,0.656977,0.640333
6,bloom_3b,0.710465,0.680594
7,bloom_7b,0.783721,0.746905
8,bloomz_560m,0.644186,0.633022
9,bloomz_1b,0.674419,0.635194
